In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from constants import *
from train_collections import *
# read the pickle file and print the data
def read_pickle_file(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f)
    # print(data)
    return data

harakat = read_pickle_file('./Delivery/diacritic2id.pickle')
print('-----------------------------')
read_pickle_file('./Delivery/diacritics.pickle')
print('-----------------------------')
lettes = read_pickle_file('./Delivery/arabic_letters.pickle')

# print(len(lettes))
# print(type(lettes))
# print(ARABIC_LETTERS)
# print(len(ARABIC_LETTERS))
print(harakat2id)
for h,id in harakat2id.items():
    print("ب"+h,id)



-----------------------------
-----------------------------
{'َ': 0, 'ً': 1, 'ُ': 2, 'ٌ': 3, 'ِ': 4, 'ٍ': 5, 'ْ': 6, 'ّ': 7, 'َّ': 8, 'ًّ': 9, 'ُّ': 10, 'ٌّ': 11, 'ِّ': 12, 'ٍّ': 13, '': 14}
بَ 0
بً 1
بُ 2
بٌ 3
بِ 4
بٍ 5
بْ 6
بّ 7
بَّ 8
بًّ 9
بُّ 10
بٌّ 11
بِّ 12
بٍّ 13
ب 14


In [6]:
import re

text = "BBARRCBAB"
re.split(r'[AC]', text)

['BB', 'RR', 'B', 'B']